# LDL decomposition rank-one update

## implementation

In [1]:
using LinearAlgebra

In [10]:
"""
    ldlDecompRank1Update(L, x)

Calculate the LDL decomposition of `LDL' + xx'` with `O(n)` computational effort, where `L` is a `n x n` lower-triangular invertible matrix, `D` is diagonal positive-definite matrix, and `x` is a `n`-dimentional vector.
The inputs `L`, `D` and `x` are modified in place.
"""
function ldlDecompRank1Update(L::UnitLowerTriangular{T1, T_L}, D::Diagonal{T2, T_D}, x::T_vec) where {T1<:Union{AbstractFloat, Complex}, T2<:AbstractFloat, T3<:Union{AbstractFloat, Complex}, T_L<:AbstractMatrix{T1}, T_D<:AbstractVector{T2}, T_vec<:AbstractVector{T3}}
    N = length(x)

    @assert N > 0
    @assert size(L) == (N,N)
    @assert size(D) == (N,N)

    T_out = promote_type(T1, T2, T3)

    for i = 1:N-1
        g = D[i,i] + abs2(x[i])
        vec_f = zeros(T_out, N)
        vec_f[i+1:end] = (D[i,i]*L[i+1:end,i] + conj(x[i])*x[i+1:end]) / g

        x[i+1:end] = sqrt(D[i,i] / g) * (x[i]*L[i+1:end,i] - x[i+1:end])
        D[i,i] = g
        L[i+1:end,i] = vec_f[i+1:end]
    end

    D[N,N] += abs2(x[N])
    return L, D
end

ldlDecompRank1Update

## test

In [20]:
let
    # Create a random Hermitian positive-define matrix.
    N = 100
    A = rand(N,N) + im*rand(N,N)
    A = A'*A + Matrix(I, N,N)
    A = Hermitian(A, :L)

    # Calculate the LDL decomposition of `A`
    BK_A = nothing
    while true
        BK_A = bunchkaufman(A)
        BK_A.p != collect(1:N) || break
    end
    L = BK_A.L
    D = Diagonal(BK_A.D)

    # Create a random vector.
    x = rand(N) + im*rand(N)

    # Create an updated matrix
    A2 = A + x*x'

    # Update the existing Cholesky decomposition.
    L2 = deepcopy(L)
    D2 = real.(deepcopy(D))
    L2, D2 = ldlDecompRank1Update(L2, D2, x)

    # Verify the result.
    A2_pred = L2*D2*L2'
    M_diff = A2_pred - A2
    println("maximum abs error: $(maximum(abs.(M_diff)))")
end

maximum abs error: 6.396513127082265e-14


## Playground

In [16]:
let
    # Create a random Hermitian positive-define matrix.
    N = 5
    A = rand(N,N) + im*rand(N,N)
    A = A'*A + Matrix(I, N,N)
    A = Hermitian(A, :L)
    println("A:")
    display(A)
    #println("eig vals of A:")
    #display(eigvals(A))

    # Calculate the LDL decomposition of `A`
    BK_A = nothing
    while true
        BK_A = bunchkaufman(A)
        BK_A.p != collect(1:N) || break
    end
    L = BK_A.L
    D = Diagonal(BK_A.D)

    # Create a random vector.
    x = rand(N) + im*rand(N)

    # Create an updated matrix
    A2 = A + x*x'
    #println("A2:")
    #display(A2)

    # Update the existing Cholesky decomposition.
    L2 = deepcopy(L)
    D2 = real.(deepcopy(D))
    L2, D2 = ldlDecompRank1Update(L2, D2, x)

    # Verify the result.
    A2_pred = L2*D2*L2'
    M_diff = A2_pred - A2
    println("L2:")
    display(L2)
    println("D2")
    display(D2)
    println("M_diff:")
    display(M_diff)
    println("maximum abs error: $(maximum(abs.(M_diff)))")
end

A:


10×10 Hermitian{ComplexF64, Matrix{ComplexF64}}:
 7.85033+0.0im        3.86151-1.68439im   …  5.02485+0.398327im
 3.86151+1.68439im     5.5352+0.0im          3.97947+0.486161im
 6.45076-0.522164im   4.43387-1.25991im      5.68534+0.136145im
 4.94801-1.076im      3.34807-1.5326im       4.52294-1.01302im
 5.75752-1.07444im    4.65414-1.99161im      5.94076-1.78748im
 5.57821+0.0603062im  4.15699-1.58373im   …  4.85934+0.34496im
 6.62006-0.623726im   4.37549-1.79302im      5.97238-0.466366im
 4.37808+1.66934im    3.09546-0.300041im     4.16749+1.04393im
  4.8423-0.344623im   4.00045-1.56344im       5.0037-0.508552im
 5.02485-0.398327im   3.97947-0.486161im     8.07691+0.0im

maximum abs error: 1.831026719408895e-15
